# Aim 2: Inpatient Mortality

## Import libraries

In [1]:
import pandas as pd

from scipy import stats

# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2

# below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML

%matplotlib inline
plt.style.use('ggplot')

## Connect to DB

In [2]:
# information used to create a database connection
sqluser = 'postgres'
dbname = 'mimic'
hostname = 'localhost'
port_number = 5434

# Connect to postgres with a copy of the MIMIC-III database
con = psycopg2.connect(dbname=dbname, user=sqluser, host=hostname, port=port_number, password='mysecretpassword')

# the below statement is prepended to queries to ensure they select from the right schema
# query_schema = 'set search_path to ' + schema_name + ';'

## Univariate Analysis - Statics

In [3]:
table1_data = []

### Age

In [4]:
sepsis_query = \
"""
SELECT
DISTINCT
PAT.subject_id,
ADM.hadm_id,
ICU.icustay_id,
ADM.hospital_expire_flag,
DATE_PART('year', AGE(ADM.admittime, PAT.dob)) as "age"
FROM patients PAT
INNER JOIN admissions ADM
ON PAT.subject_id = ADM.subject_id
LEFT JOIN icustays ICU
ON ICU.subject_id = ADM.subject_id AND ICU.hadm_id = ADM.hadm_id
INNER JOIN diagnoses_icd DIA_ICD
ON DIA_ICD.subject_id = ADM.subject_id AND DIA_ICD.hadm_id = ADM.hadm_id AND DIA_ICD.icd9_code in ('99591', '99592', '78552')
WHERE DATE_PART('year', AGE(ADM.admittime, PAT.dob)) > 18
;
"""

sepsis_patients = pd.read_sql_query(sepsis_query, con)
sepsis_patients

,subject_id,hadm_id,icustay_id,hospital_expire_flag,age
0,21,111970,216859.0,1,87.0
1,38,185910,248910.0,0,75.0
2,61,189535,217135.0,1,55.0
3,62,116009,216609.0,0,68.0
4,64,172056,232593.0,0,26.0
...,...,...,...,...,...
6125,99865,189346,259297.0,1,77.0
6126,99865,189346,284772.0,1,77.0
6127,99912,189380,236183.0,0,84.0
6128,99985,176670,279638.0,0,53.0


In [5]:
patients_he_1 = sepsis_patients[sepsis_patients.hospital_expire_flag==1].age
patients_he_0 = sepsis_patients[sepsis_patients.hospital_expire_flag==0].age
t_value, p_value = stats.ttest_ind(
        patients_he_1,
        patients_he_0
)

table1_data.append(
    [
        'Age (in years)',
        patients_he_1.mean(),
        patients_he_0.mean(),
        t_value,
        p_value
    ]
)
table1_data

[['Age (in years)',
  84.91874376869391,
  79.14403491755577,
  3.422264391851681,
  0.0006250811391058722]]

### Ethnicity

In [6]:
sepsis_query = \
"""
SELECT
DISTINCT
PAT.subject_id,
ADM.hadm_id,
ICU.icustay_id,
ADM.hospital_expire_flag,
ADM.ethnicity as "ethnicity"
FROM patients PAT
INNER JOIN admissions ADM
ON PAT.subject_id = ADM.subject_id
LEFT JOIN icustays ICU
ON ICU.subject_id = ADM.subject_id AND ICU.hadm_id = ADM.hadm_id
INNER JOIN diagnoses_icd DIA_ICD
ON DIA_ICD.subject_id = ADM.subject_id AND DIA_ICD.hadm_id = ADM.hadm_id AND DIA_ICD.icd9_code in ('99591', '99592', '78552')
WHERE DATE_PART('year', AGE(ADM.admittime, PAT.dob)) > 18
;
"""

sepsis_patients = pd.read_sql_query(sepsis_query, con)
sepsis_patients

,subject_id,hadm_id,icustay_id,hospital_expire_flag,ethnicity
0,21,111970,216859.0,1,WHITE
1,38,185910,248910.0,0,WHITE
2,61,189535,217135.0,1,WHITE
3,62,116009,216609.0,0,PATIENT DECLINED TO ANSWER
4,64,172056,232593.0,0,BLACK/AFRICAN AMERICAN
...,...,...,...,...,...
6125,99865,189346,259297.0,1,WHITE
6126,99865,189346,284772.0,1,WHITE
6127,99912,189380,236183.0,0,WHITE
6128,99985,176670,279638.0,0,WHITE


In [7]:
def categorize_ethnicity(ethnicity):
    if 'ASIAN' in ethnicity:
        ethnicity = 'ASIAN'
    elif 'WHITE' in ethnicity:
        ethnicity = 'WHITE'
    elif 'HISPANIC' in ethnicity:
        ethnicity = 'HISPANIC/LATINO'
    elif 'BLACK' in ethnicity:
        ethnicity = 'BLACK'
    else:
        ethnicity = 'OTHER'
    return ethnicity

sepsis_patients['ethnicity'] = sepsis_patients.ethnicity.apply(categorize_ethnicity)
sepsis_patients

,subject_id,hadm_id,icustay_id,hospital_expire_flag,ethnicity
0,21,111970,216859.0,1,WHITE
1,38,185910,248910.0,0,WHITE
2,61,189535,217135.0,1,WHITE
3,62,116009,216609.0,0,OTHER
4,64,172056,232593.0,0,BLACK
...,...,...,...,...,...
6125,99865,189346,259297.0,1,WHITE
6126,99865,189346,284772.0,1,WHITE
6127,99912,189380,236183.0,0,WHITE
6128,99985,176670,279638.0,0,WHITE


In [8]:
for ethnicity in ['ASIAN', 'WHITE', 'HISPANIC/LATINO', 'BLACK', 'OTHER']:
    patients_he_1 = [1 if (e == ethnicity) else 0 for e in sepsis_patients[sepsis_patients.hospital_expire_flag==1].ethnicity]
    patients_he_0 = [1 if (e == ethnicity) else 0 for e in sepsis_patients[sepsis_patients.hospital_expire_flag==0].ethnicity]
    t_value, p_value = stats.ttest_ind(
        patients_he_1,
        patients_he_0
    )

    table1_data.append(
        [
            'Ethnicity = ' + str(ethnicity),
            sum(patients_he_1),
            sum(patients_he_0),
            t_value,
            p_value
        ]
    )
table1_data

[['Age (in years)',
  84.91874376869391,
  79.14403491755577,
  3.422264391851681,
  0.0006250811391058722],
 ['Ethnicity = ASIAN', 56, 138, -1.1638473949703363, 0.24453117281890696],
 ['Ethnicity = WHITE', 1460, 3047, -0.9182400239799813, 0.358529357829805],
 ['Ethnicity = HISPANIC/LATINO',
  56,
  143,
  -1.400987264851608,
  0.1612685182515495],
 ['Ethnicity = BLACK', 182, 458, -2.443116508157017, 0.014589070828960967],
 ['Ethnicity = OTHER', 252, 338, 5.450995727524083, 5.204359840241054e-08]]

### First Care Unit

In [9]:
sepsis_query = \
"""
SELECT
DISTINCT
PAT.subject_id,
ADM.hadm_id,
ICU.icustay_id,
ADM.hospital_expire_flag,
ICU.first_careunit as "first_careunit"
FROM patients PAT
INNER JOIN admissions ADM
ON PAT.subject_id = ADM.subject_id
LEFT JOIN icustays ICU
ON ICU.subject_id = ADM.subject_id AND ICU.hadm_id = ADM.hadm_id
INNER JOIN diagnoses_icd DIA_ICD
ON DIA_ICD.subject_id = ADM.subject_id AND DIA_ICD.hadm_id = ADM.hadm_id AND DIA_ICD.icd9_code in ('99591', '99592', '78552')
WHERE DATE_PART('year', AGE(ADM.admittime, PAT.dob)) > 18
;
"""

sepsis_patients = pd.read_sql_query(sepsis_query, con)
sepsis_patients

,subject_id,hadm_id,icustay_id,hospital_expire_flag,first_careunit
0,21,111970,216859.0,1,MICU
1,38,185910,248910.0,0,CCU
2,61,189535,217135.0,1,MICU
3,62,116009,216609.0,0,CCU
4,64,172056,232593.0,0,MICU
...,...,...,...,...,...
6125,99865,189346,259297.0,1,SICU
6126,99865,189346,284772.0,1,SICU
6127,99912,189380,236183.0,0,MICU
6128,99985,176670,279638.0,0,MICU


In [10]:
for careunit in ['CCU', 'MICU', 'CSRU', 'SICU']:
    patients_he_1 = [1 if (fcu == careunit) else 0 for fcu in sepsis_patients[sepsis_patients.hospital_expire_flag==1].first_careunit]
    patients_he_0 = [1 if (fcu == careunit) else 0 for fcu in sepsis_patients[sepsis_patients.hospital_expire_flag==0].first_careunit]
    t_value, p_value = stats.ttest_ind(
        patients_he_1,
        patients_he_0
    )

    table1_data.append(
        [
            'Careunit = ' + str(careunit),
            sum(patients_he_1),
            sum(patients_he_0),
            t_value,
            p_value
        ]
    )
table1_data

[['Age (in years)',
  84.91874376869391,
  79.14403491755577,
  3.422264391851681,
  0.0006250811391058722],
 ['Ethnicity = ASIAN', 56, 138, -1.1638473949703363, 0.24453117281890696],
 ['Ethnicity = WHITE', 1460, 3047, -0.9182400239799813, 0.358529357829805],
 ['Ethnicity = HISPANIC/LATINO',
  56,
  143,
  -1.400987264851608,
  0.1612685182515495],
 ['Ethnicity = BLACK', 182, 458, -2.443116508157017, 0.014589070828960967],
 ['Ethnicity = OTHER', 252, 338, 5.450995727524083, 5.204359840241054e-08],
 ['Careunit = CCU', 200, 356, 1.7112908156968956, 0.08707804587029497],
 ['Careunit = MICU', 1305, 2770, -1.6443796349501218, 0.10014910264828243],
 ['Careunit = CSRU', 86, 143, 1.5878573610397497, 0.1123701880566653],
 ['Careunit = SICU', 274, 579, -0.4040687714105425, 0.6861762488200903]]

### Marital Status

In [11]:
sepsis_query = \
"""
SELECT
DISTINCT
PAT.subject_id,
ADM.hadm_id,
ICU.icustay_id,
ADM.hospital_expire_flag,
ADM.marital_status as "marital_status"
FROM patients PAT
INNER JOIN admissions ADM
ON PAT.subject_id = ADM.subject_id
LEFT JOIN icustays ICU
ON ICU.subject_id = ADM.subject_id AND ICU.hadm_id = ADM.hadm_id
INNER JOIN diagnoses_icd DIA_ICD
ON DIA_ICD.subject_id = ADM.subject_id AND DIA_ICD.hadm_id = ADM.hadm_id AND DIA_ICD.icd9_code in ('99591', '99592', '78552')
WHERE DATE_PART('year', AGE(ADM.admittime, PAT.dob)) > 18
;
"""

sepsis_patients = pd.read_sql_query(sepsis_query, con)
sepsis_patients

,subject_id,hadm_id,icustay_id,hospital_expire_flag,marital_status
0,21,111970,216859.0,1,MARRIED
1,38,185910,248910.0,0,WIDOWED
2,61,189535,217135.0,1,MARRIED
3,62,116009,216609.0,0,MARRIED
4,64,172056,232593.0,0,SINGLE
...,...,...,...,...,...
6125,99865,189346,259297.0,1,DIVORCED
6126,99865,189346,284772.0,1,DIVORCED
6127,99912,189380,236183.0,0,MARRIED
6128,99985,176670,279638.0,0,MARRIED


In [12]:
def categorize_maritalstatus(maritalstatus):
    if maritalstatus:
        if 'DIVORCED' in maritalstatus:
            maritalstatus = 'SINGLE/SEPARATED/DIVORCED/WIDOW'
        elif 'SEPARATED' in maritalstatus:
            maritalstatus = 'SINGLE/SEPARATED/DIVORCED/WIDOW'
        elif 'SINGLE' in maritalstatus:
            maritalstatus = 'SINGLE/SEPARATED/DIVORCED/WIDOW'
        elif 'WIDOWED' in maritalstatus:
            maritalstatus = 'SINGLE/SEPARATED/DIVORCED/WIDOW'
        elif 'LIFE PARTNER' in maritalstatus:
            maritalstatus = 'MARRIED'
        elif 'MARRIED' in maritalstatus:
            maritalstatus = 'MARRIED'
        else:
            maritalstatus = 'OTHER'
    else:
        maritalstatus = 'OTHER'
    return maritalstatus

sepsis_patients['marital_status'] = sepsis_patients.marital_status.apply(categorize_maritalstatus)
sepsis_patients

,subject_id,hadm_id,icustay_id,hospital_expire_flag,marital_status
0,21,111970,216859.0,1,MARRIED
1,38,185910,248910.0,0,SINGLE/SEPARATED/DIVORCED/WIDOW
2,61,189535,217135.0,1,MARRIED
3,62,116009,216609.0,0,MARRIED
4,64,172056,232593.0,0,SINGLE/SEPARATED/DIVORCED/WIDOW
...,...,...,...,...,...
6125,99865,189346,259297.0,1,SINGLE/SEPARATED/DIVORCED/WIDOW
6126,99865,189346,284772.0,1,SINGLE/SEPARATED/DIVORCED/WIDOW
6127,99912,189380,236183.0,0,MARRIED
6128,99985,176670,279638.0,0,MARRIED


In [13]:
for marital_status in ['SINGLE/SEPARATED/DIVORCED/WIDOW', 'MARRIED', 'OTHER']:
    patients_he_1 = [1 if (ms == marital_status) else 0 for ms in sepsis_patients[sepsis_patients.hospital_expire_flag==1].marital_status]
    patients_he_0 = [1 if (ms == marital_status) else 0 for ms in sepsis_patients[sepsis_patients.hospital_expire_flag==0].marital_status]
    t_value, p_value = stats.ttest_ind(
        patients_he_1,
        patients_he_0
    )

    table1_data.append(
        [
            'Maritalstatus = ' + str(marital_status),
            sum(patients_he_1),
            sum(patients_he_0),
            t_value,
            p_value
        ]
    )
table1_data

[['Age (in years)',
  84.91874376869391,
  79.14403491755577,
  3.422264391851681,
  0.0006250811391058722],
 ['Ethnicity = ASIAN', 56, 138, -1.1638473949703363, 0.24453117281890696],
 ['Ethnicity = WHITE', 1460, 3047, -0.9182400239799813, 0.358529357829805],
 ['Ethnicity = HISPANIC/LATINO',
  56,
  143,
  -1.400987264851608,
  0.1612685182515495],
 ['Ethnicity = BLACK', 182, 458, -2.443116508157017, 0.014589070828960967],
 ['Ethnicity = OTHER', 252, 338, 5.450995727524083, 5.204359840241054e-08],
 ['Careunit = CCU', 200, 356, 1.7112908156968956, 0.08707804587029497],
 ['Careunit = MICU', 1305, 2770, -1.6443796349501218, 0.10014910264828243],
 ['Careunit = CSRU', 86, 143, 1.5878573610397497, 0.1123701880566653],
 ['Careunit = SICU', 274, 579, -0.4040687714105425, 0.6861762488200903],
 ['Maritalstatus = SINGLE/SEPARATED/DIVORCED/WIDOW',
  947,
  2093,
  -2.604468178460079,
  0.009223914046205005],
 ['Maritalstatus = MARRIED',
  943,
  1868,
  1.2629915794872069,
  0.20664025917200965],


### Insurance

In [14]:
sepsis_query = \
"""
SELECT
DISTINCT
PAT.subject_id,
ADM.hadm_id,
ICU.icustay_id,
ADM.hospital_expire_flag,
ADM.insurance as "insurance"
FROM patients PAT
INNER JOIN admissions ADM
ON PAT.subject_id = ADM.subject_id
LEFT JOIN icustays ICU
ON ICU.subject_id = ADM.subject_id AND ICU.hadm_id = ADM.hadm_id
INNER JOIN diagnoses_icd DIA_ICD
ON DIA_ICD.subject_id = ADM.subject_id AND DIA_ICD.hadm_id = ADM.hadm_id AND DIA_ICD.icd9_code in ('99591', '99592', '78552')
WHERE DATE_PART('year', AGE(ADM.admittime, PAT.dob)) > 18
;
"""

sepsis_patients = pd.read_sql_query(sepsis_query, con)
sepsis_patients

,subject_id,hadm_id,icustay_id,hospital_expire_flag,insurance
0,21,111970,216859.0,1,Medicare
1,38,185910,248910.0,0,Medicare
2,61,189535,217135.0,1,Private
3,62,116009,216609.0,0,Medicare
4,64,172056,232593.0,0,Medicaid
...,...,...,...,...,...
6125,99865,189346,259297.0,1,Medicare
6126,99865,189346,284772.0,1,Medicare
6127,99912,189380,236183.0,0,Medicare
6128,99985,176670,279638.0,0,Private


In [15]:
def categorize_insurance(insurance):
    if insurance:
        if 'Government' in insurance:
            insurance = 'GOVERNMENT'
        elif 'Private' in insurance:
            insurance = 'PRIVATE'
        elif 'Self Pay' in insurance:
            insurance = 'SELF PAY'
        elif 'Medicare' in insurance:
            insurance = 'MEDICARE/MEDICAID'
        elif 'Medicaid' in insurance:
            insurance = 'MEDICARE/MEDICAID'
        else:
            insurance = 'OTHER'
    else:
        insurance = 'OTHER'
    return insurance

sepsis_patients['insurance'] = sepsis_patients.insurance.apply(categorize_insurance)
sepsis_patients

,subject_id,hadm_id,icustay_id,hospital_expire_flag,insurance
0,21,111970,216859.0,1,MEDICARE/MEDICAID
1,38,185910,248910.0,0,MEDICARE/MEDICAID
2,61,189535,217135.0,1,PRIVATE
3,62,116009,216609.0,0,MEDICARE/MEDICAID
4,64,172056,232593.0,0,MEDICARE/MEDICAID
...,...,...,...,...,...
6125,99865,189346,259297.0,1,MEDICARE/MEDICAID
6126,99865,189346,284772.0,1,MEDICARE/MEDICAID
6127,99912,189380,236183.0,0,MEDICARE/MEDICAID
6128,99985,176670,279638.0,0,PRIVATE


In [16]:
for insurance in ['MEDICARE/MEDICAID', 'PRIVATE', 'GOVERNMENT', 'SELF PAY']:
    patients_he_1 = [1 if (i == insurance) else 0 for i in sepsis_patients[sepsis_patients.hospital_expire_flag==1].insurance]
    patients_he_0 = [1 if (i == insurance) else 0 for i in sepsis_patients[sepsis_patients.hospital_expire_flag==0].insurance]
    t_value, p_value = stats.ttest_ind(
        patients_he_1,
        patients_he_0
    )

    table1_data.append(
        [
            'Insurance = ' + str(insurance),
            sum(patients_he_1),
            sum(patients_he_0),
            t_value,
            p_value
        ]
    )
table1_data

[['Age (in years)',
  84.91874376869391,
  79.14403491755577,
  3.422264391851681,
  0.0006250811391058722],
 ['Ethnicity = ASIAN', 56, 138, -1.1638473949703363, 0.24453117281890696],
 ['Ethnicity = WHITE', 1460, 3047, -0.9182400239799813, 0.358529357829805],
 ['Ethnicity = HISPANIC/LATINO',
  56,
  143,
  -1.400987264851608,
  0.1612685182515495],
 ['Ethnicity = BLACK', 182, 458, -2.443116508157017, 0.014589070828960967],
 ['Ethnicity = OTHER', 252, 338, 5.450995727524083, 5.204359840241054e-08],
 ['Careunit = CCU', 200, 356, 1.7112908156968956, 0.08707804587029497],
 ['Careunit = MICU', 1305, 2770, -1.6443796349501218, 0.10014910264828243],
 ['Careunit = CSRU', 86, 143, 1.5878573610397497, 0.1123701880566653],
 ['Careunit = SICU', 274, 579, -0.4040687714105425, 0.6861762488200903],
 ['Maritalstatus = SINGLE/SEPARATED/DIVORCED/WIDOW',
  947,
  2093,
  -2.604468178460079,
  0.009223914046205005],
 ['Maritalstatus = MARRIED',
  943,
  1868,
  1.2629915794872069,
  0.20664025917200965],


### Admission Type

In [17]:
sepsis_query = \
"""
SELECT
DISTINCT
PAT.subject_id,
ADM.hadm_id,
ICU.icustay_id,
ADM.hospital_expire_flag,
ADM.admission_type as "admission_type"
FROM patients PAT
INNER JOIN admissions ADM
ON PAT.subject_id = ADM.subject_id
LEFT JOIN icustays ICU
ON ICU.subject_id = ADM.subject_id AND ICU.hadm_id = ADM.hadm_id
INNER JOIN diagnoses_icd DIA_ICD
ON DIA_ICD.subject_id = ADM.subject_id AND DIA_ICD.hadm_id = ADM.hadm_id AND DIA_ICD.icd9_code in ('99591', '99592', '78552')
WHERE DATE_PART('year', AGE(ADM.admittime, PAT.dob)) > 18
;
"""

sepsis_patients = pd.read_sql_query(sepsis_query, con)
sepsis_patients

,subject_id,hadm_id,icustay_id,hospital_expire_flag,admission_type
0,21,111970,216859.0,1,EMERGENCY
1,38,185910,248910.0,0,EMERGENCY
2,61,189535,217135.0,1,EMERGENCY
3,62,116009,216609.0,0,EMERGENCY
4,64,172056,232593.0,0,EMERGENCY
...,...,...,...,...,...
6125,99865,189346,259297.0,1,ELECTIVE
6126,99865,189346,284772.0,1,ELECTIVE
6127,99912,189380,236183.0,0,EMERGENCY
6128,99985,176670,279638.0,0,EMERGENCY


In [18]:
for admission_type in ['URGENT', 'ELECTIVE', 'EMERGENCY']:
    patients_he_1 = [1 if (at == admission_type) else 0 for at in sepsis_patients[sepsis_patients.hospital_expire_flag==1].admission_type]
    patients_he_0 = [1 if (at == admission_type) else 0 for at in sepsis_patients[sepsis_patients.hospital_expire_flag==0].admission_type]
    t_value, p_value = stats.ttest_ind(
        patients_he_1,
        patients_he_0
    )

    table1_data.append(
        [
            'Admission Type = ' + str(admission_type),
            sum(patients_he_1),
            sum(patients_he_0),
            t_value,
            p_value
        ]
    )
table1_data

[['Age (in years)',
  84.91874376869391,
  79.14403491755577,
  3.422264391851681,
  0.0006250811391058722],
 ['Ethnicity = ASIAN', 56, 138, -1.1638473949703363, 0.24453117281890696],
 ['Ethnicity = WHITE', 1460, 3047, -0.9182400239799813, 0.358529357829805],
 ['Ethnicity = HISPANIC/LATINO',
  56,
  143,
  -1.400987264851608,
  0.1612685182515495],
 ['Ethnicity = BLACK', 182, 458, -2.443116508157017, 0.014589070828960967],
 ['Ethnicity = OTHER', 252, 338, 5.450995727524083, 5.204359840241054e-08],
 ['Careunit = CCU', 200, 356, 1.7112908156968956, 0.08707804587029497],
 ['Careunit = MICU', 1305, 2770, -1.6443796349501218, 0.10014910264828243],
 ['Careunit = CSRU', 86, 143, 1.5878573610397497, 0.1123701880566653],
 ['Careunit = SICU', 274, 579, -0.4040687714105425, 0.6861762488200903],
 ['Maritalstatus = SINGLE/SEPARATED/DIVORCED/WIDOW',
  947,
  2093,
  -2.604468178460079,
  0.009223914046205005],
 ['Maritalstatus = MARRIED',
  943,
  1868,
  1.2629915794872069,
  0.20664025917200965],


### Admission Location

In [19]:
sepsis_query = \
"""
SELECT
DISTINCT
PAT.subject_id,
ADM.hadm_id,
ICU.icustay_id,
ADM.hospital_expire_flag,
ADM.admission_location as "admission_location"
FROM patients PAT
INNER JOIN admissions ADM
ON PAT.subject_id = ADM.subject_id
LEFT JOIN icustays ICU
ON ICU.subject_id = ADM.subject_id AND ICU.hadm_id = ADM.hadm_id
INNER JOIN diagnoses_icd DIA_ICD
ON DIA_ICD.subject_id = ADM.subject_id AND DIA_ICD.hadm_id = ADM.hadm_id AND DIA_ICD.icd9_code in ('99591', '99592', '78552')
WHERE DATE_PART('year', AGE(ADM.admittime, PAT.dob)) > 18
;
"""

sepsis_patients = pd.read_sql_query(sepsis_query, con)
sepsis_patients

,subject_id,hadm_id,icustay_id,hospital_expire_flag,admission_location
0,21,111970,216859.0,1,EMERGENCY ROOM ADMIT
1,38,185910,248910.0,0,TRANSFER FROM HOSP/EXTRAM
2,61,189535,217135.0,1,CLINIC REFERRAL/PREMATURE
3,62,116009,216609.0,0,EMERGENCY ROOM ADMIT
4,64,172056,232593.0,0,EMERGENCY ROOM ADMIT
...,...,...,...,...,...
6125,99865,189346,259297.0,1,PHYS REFERRAL/NORMAL DELI
6126,99865,189346,284772.0,1,PHYS REFERRAL/NORMAL DELI
6127,99912,189380,236183.0,0,EMERGENCY ROOM ADMIT
6128,99985,176670,279638.0,0,EMERGENCY ROOM ADMIT


In [20]:
def categorize_admission_location(admission_location):
    if admission_location:
        if 'REFERRAL' in admission_location:
            admission_location = 'REFERRAL'
        elif 'EMERGENCY' in admission_location:
            admission_location = 'EMERGENCY'
        elif 'TRANSFER' in admission_location:
            admission_location = 'TRANSFER'
        elif 'TRSF' in admission_location:
            admission_location = 'TRANSFER'
        else:
            admission_location = 'OTHER'
    else:
        admission_location = 'OTHER'
    return admission_location

sepsis_patients['admission_location'] = sepsis_patients.admission_location.apply(categorize_admission_location)
sepsis_patients

,subject_id,hadm_id,icustay_id,hospital_expire_flag,admission_location
0,21,111970,216859.0,1,EMERGENCY
1,38,185910,248910.0,0,TRANSFER
2,61,189535,217135.0,1,REFERRAL
3,62,116009,216609.0,0,EMERGENCY
4,64,172056,232593.0,0,EMERGENCY
...,...,...,...,...,...
6125,99865,189346,259297.0,1,REFERRAL
6126,99865,189346,284772.0,1,REFERRAL
6127,99912,189380,236183.0,0,EMERGENCY
6128,99985,176670,279638.0,0,EMERGENCY


In [21]:
for admission_location in ['REFERRAL', 'EMERGENCY', 'TRANSFER']:
    patients_he_1 = [1 if (al == admission_location) else 0 for al in sepsis_patients[sepsis_patients.hospital_expire_flag==1].admission_location]
    patients_he_0 = [1 if (al == admission_location) else 0 for al in sepsis_patients[sepsis_patients.hospital_expire_flag==0].admission_location]
    t_value, p_value = stats.ttest_ind(
        patients_he_1,
        patients_he_0
    )

    table1_data.append(
        [
            'Admission Location = ' + str(admission_location),
            sum(patients_he_1),
            sum(patients_he_0),
            t_value,
            p_value
        ]
    )
table1_data

[['Age (in years)',
  84.91874376869391,
  79.14403491755577,
  3.422264391851681,
  0.0006250811391058722],
 ['Ethnicity = ASIAN', 56, 138, -1.1638473949703363, 0.24453117281890696],
 ['Ethnicity = WHITE', 1460, 3047, -0.9182400239799813, 0.358529357829805],
 ['Ethnicity = HISPANIC/LATINO',
  56,
  143,
  -1.400987264851608,
  0.1612685182515495],
 ['Ethnicity = BLACK', 182, 458, -2.443116508157017, 0.014589070828960967],
 ['Ethnicity = OTHER', 252, 338, 5.450995727524083, 5.204359840241054e-08],
 ['Careunit = CCU', 200, 356, 1.7112908156968956, 0.08707804587029497],
 ['Careunit = MICU', 1305, 2770, -1.6443796349501218, 0.10014910264828243],
 ['Careunit = CSRU', 86, 143, 1.5878573610397497, 0.1123701880566653],
 ['Careunit = SICU', 274, 579, -0.4040687714105425, 0.6861762488200903],
 ['Maritalstatus = SINGLE/SEPARATED/DIVORCED/WIDOW',
  947,
  2093,
  -2.604468178460079,
  0.009223914046205005],
 ['Maritalstatus = MARRIED',
  943,
  1868,
  1.2629915794872069,
  0.20664025917200965],


### Diagnosis

In [22]:
sepsis_query = \
"""
SELECT
DISTINCT
PAT.subject_id,
ADM.hadm_id,
ICU.icustay_id,
ADM.hospital_expire_flag,
ADM.diagnosis as "diagnosis"
FROM patients PAT
INNER JOIN admissions ADM
ON PAT.subject_id = ADM.subject_id
LEFT JOIN icustays ICU
ON ICU.subject_id = ADM.subject_id AND ICU.hadm_id = ADM.hadm_id
INNER JOIN diagnoses_icd DIA_ICD
ON DIA_ICD.subject_id = ADM.subject_id AND DIA_ICD.hadm_id = ADM.hadm_id AND DIA_ICD.icd9_code in ('99591', '99592', '78552')
WHERE DATE_PART('year', AGE(ADM.admittime, PAT.dob)) > 18
;
"""

sepsis_patients = pd.read_sql_query(sepsis_query, con)
sepsis_patients

,subject_id,hadm_id,icustay_id,hospital_expire_flag,diagnosis
0,21,111970,216859.0,1,SEPSIS
1,38,185910,248910.0,0,ACUTE MYOCARDIAL INFARCTION-SEPSIS
2,61,189535,217135.0,1,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA
3,62,116009,216609.0,0,"SEPSIS,URINARY TRACT INFECTION"
4,64,172056,232593.0,0,FUNGAL MENINGITIS
...,...,...,...,...,...
6125,99865,189346,259297.0,1,HEPATOCELLULAR CARCINOMA/SDA
6126,99865,189346,284772.0,1,HEPATOCELLULAR CARCINOMA/SDA
6127,99912,189380,236183.0,0,PNEUMONIA
6128,99985,176670,279638.0,0,FEVER


In [23]:
def categorize_diagnosis(diagnosis):
    if diagnosis:
        if 'SEPSIS' in diagnosis:
            diagnosis = 'SEPSIS'
        else:
            diagnosis = 'OTHER'
    else:
        diagnosis = 'OTHER'
    return diagnosis

sepsis_patients['diagnosis'] = sepsis_patients.diagnosis.apply(categorize_diagnosis)
sepsis_patients

,subject_id,hadm_id,icustay_id,hospital_expire_flag,diagnosis
0,21,111970,216859.0,1,SEPSIS
1,38,185910,248910.0,0,SEPSIS
2,61,189535,217135.0,1,OTHER
3,62,116009,216609.0,0,SEPSIS
4,64,172056,232593.0,0,OTHER
...,...,...,...,...,...
6125,99865,189346,259297.0,1,OTHER
6126,99865,189346,284772.0,1,OTHER
6127,99912,189380,236183.0,0,OTHER
6128,99985,176670,279638.0,0,OTHER


In [24]:
for diagnosis in sepsis_patients.diagnosis.unique():
    patients_he_1 = [1 if (d == diagnosis) else 0 for d in sepsis_patients[sepsis_patients.hospital_expire_flag==1].diagnosis]
    patients_he_0 = [1 if (d == diagnosis) else 0 for d in sepsis_patients[sepsis_patients.hospital_expire_flag==0].diagnosis]
    t_value, p_value = stats.ttest_ind(
        patients_he_1,
        patients_he_0
    )

    table1_data.append(
        [
            'Diagnosis = ' + str(diagnosis),
            sum(patients_he_1),
            sum(patients_he_0),
            t_value,
            p_value
        ]
    )
table1_data

[['Age (in years)',
  84.91874376869391,
  79.14403491755577,
  3.422264391851681,
  0.0006250811391058722],
 ['Ethnicity = ASIAN', 56, 138, -1.1638473949703363, 0.24453117281890696],
 ['Ethnicity = WHITE', 1460, 3047, -0.9182400239799813, 0.358529357829805],
 ['Ethnicity = HISPANIC/LATINO',
  56,
  143,
  -1.400987264851608,
  0.1612685182515495],
 ['Ethnicity = BLACK', 182, 458, -2.443116508157017, 0.014589070828960967],
 ['Ethnicity = OTHER', 252, 338, 5.450995727524083, 5.204359840241054e-08],
 ['Careunit = CCU', 200, 356, 1.7112908156968956, 0.08707804587029497],
 ['Careunit = MICU', 1305, 2770, -1.6443796349501218, 0.10014910264828243],
 ['Careunit = CSRU', 86, 143, 1.5878573610397497, 0.1123701880566653],
 ['Careunit = SICU', 274, 579, -0.4040687714105425, 0.6861762488200903],
 ['Maritalstatus = SINGLE/SEPARATED/DIVORCED/WIDOW',
  947,
  2093,
  -2.604468178460079,
  0.009223914046205005],
 ['Maritalstatus = MARRIED',
  943,
  1868,
  1.2629915794872069,
  0.20664025917200965],


## Summary - Statics

In [25]:
table1_df = pd.DataFrame(table1_data, columns=['Attribute', 'Hospital Expired == 1', 'Hospital Expired == 0', 't_value', 'p_value'])
table1_df

,Attribute,Hospital Expired == 1,Hospital Expired == 0,t_value,p_value
0,Age (in years),84.918744,79.144035,3.422264,6.250811e-04
1,Ethnicity = ASIAN,56.000000,138.000000,-1.163847,2.445312e-01
2,Ethnicity = WHITE,1460.000000,3047.000000,-0.918240,3.585294e-01
3,Ethnicity = HISPANIC/LATINO,56.000000,143.000000,-1.400987,1.612685e-01
4,Ethnicity = BLACK,182.000000,458.000000,-2.443117,1.458907e-02
5,Ethnicity = OTHER,252.000000,338.000000,5.450996,5.204360e-08
6,Careunit = CCU,200.000000,356.000000,1.711291,8.707805e-02
7,Careunit = MICU,1305.000000,2770.000000,-1.644380,1.001491e-01
8,Careunit = CSRU,86.000000,143.000000,1.587857,1.123702e-01
9,Careunit = SICU,274.000000,579.000000,-0.404069,6.861762e-01


## Univariate Analysis - ICU Stay

In [26]:
table2_data = []

### First Hospital Stay

In [27]:
sepsis_query = \
"""
SELECT
DISTINCT
PAT.subject_id,
ADM.hadm_id,
ICU.icustay_id,
ICU_D.first_hosp_stay,
ADM.hospital_expire_flag
FROM patients PAT
INNER JOIN admissions ADM
ON PAT.subject_id = ADM.subject_id
LEFT JOIN icustays ICU
ON ICU.subject_id = ADM.subject_id AND ICU.hadm_id = ADM.hadm_id
LEFT JOIN icustay_detail ICU_D
ON ICU_D.icustay_id = ICU.icustay_id
INNER JOIN diagnoses_icd DIA_ICD
ON DIA_ICD.subject_id = ADM.subject_id AND DIA_ICD.hadm_id = ADM.hadm_id AND DIA_ICD.icd9_code in ('99591', '99592', '78552')
WHERE DATE_PART('year', AGE(ADM.admittime, PAT.dob)) > 18
;
"""

sepsis_patients = pd.read_sql_query(sepsis_query, con)
sepsis_patients

,subject_id,hadm_id,icustay_id,first_hosp_stay,hospital_expire_flag
0,21,111970,216859.0,False,1
1,38,185910,248910.0,True,0
2,61,189535,217135.0,False,1
3,62,116009,216609.0,True,0
4,64,172056,232593.0,True,0
...,...,...,...,...,...
6125,99865,189346,259297.0,True,1
6126,99865,189346,284772.0,True,1
6127,99912,189380,236183.0,True,0
6128,99985,176670,279638.0,True,0


In [28]:
for first_hosp_stay in [True, False]:
    patients_he_1 = [1 if (fhs == first_hosp_stay) else 0 for fhs in sepsis_patients[sepsis_patients.hospital_expire_flag==1].first_hosp_stay]
    patients_he_0 = [1 if (fhs == first_hosp_stay) else 0 for fhs in sepsis_patients[sepsis_patients.hospital_expire_flag==0].first_hosp_stay]
    t_value, p_value = stats.ttest_ind(
        patients_he_1,
        patients_he_0
    )

    table2_data.append(
        [
            'First Hospital Stay = ' + str(first_hosp_stay),
            sum(patients_he_1),
            sum(patients_he_0),
            t_value,
            p_value
        ]
    )
table2_data

[['First Hospital Stay = True',
  1329,
  2717,
  0.28581833163806625,
  0.7750269276096289],
 ['First Hospital Stay = False',
  673,
  1396,
  -0.23402245384060752,
  0.8149753645772002]]

### LOS Hospital

In [29]:
sepsis_query = \
"""
SELECT
DISTINCT
PAT.subject_id,
ADM.hadm_id,
ICU.icustay_id,
ICU_D.los_hospital,
ADM.hospital_expire_flag
FROM patients PAT
INNER JOIN admissions ADM
ON PAT.subject_id = ADM.subject_id
LEFT JOIN icustays ICU
ON ICU.subject_id = ADM.subject_id AND ICU.hadm_id = ADM.hadm_id
LEFT JOIN icustay_detail ICU_D
ON ICU_D.icustay_id = ICU.icustay_id
INNER JOIN diagnoses_icd DIA_ICD
ON DIA_ICD.subject_id = ADM.subject_id AND DIA_ICD.hadm_id = ADM.hadm_id AND DIA_ICD.icd9_code in ('99591', '99592', '78552')
WHERE DATE_PART('year', AGE(ADM.admittime, PAT.dob)) > 18
;
"""

sepsis_patients = pd.read_sql_query(sepsis_query, con)
sepsis_patients

,subject_id,hadm_id,icustay_id,los_hospital,hospital_expire_flag
0,21,111970,216859.0,8.220833,1
1,38,185910,248910.0,25.459722,0
2,61,189535,217135.0,29.307639,1
3,62,116009,216609.0,4.632639,0
4,64,172056,232593.0,15.150000,0
...,...,...,...,...,...
6125,99865,189346,259297.0,19.563194,1
6126,99865,189346,284772.0,19.563194,1
6127,99912,189380,236183.0,13.899306,0
6128,99985,176670,279638.0,16.595833,0


In [30]:
patients_he_1 = sepsis_patients[(sepsis_patients.hospital_expire_flag==1) & sepsis_patients.los_hospital.notna()].los_hospital
patients_he_0 = sepsis_patients[(sepsis_patients.hospital_expire_flag==0) & sepsis_patients.los_hospital.notna()].los_hospital
t_value, p_value = stats.ttest_ind(
        patients_he_1,
        patients_he_0
)

table2_data.append(
    [
        'LOS Hospital',
        patients_he_1.mean(),
        patients_he_0.mean(),
        t_value,
        p_value
    ]
)
table2_data

[['First Hospital Stay = True',
  1329,
  2717,
  0.28581833163806625,
  0.7750269276096289],
 ['First Hospital Stay = False',
  673,
  1396,
  -0.23402245384060752,
  0.8149753645772002],
 ['LOS Hospital',
  18.229534354534355,
  18.24540160601886,
  -0.027154426541734923,
  0.9783374510146332]]

### First ICU Stay

In [31]:
sepsis_query = \
"""
SELECT
DISTINCT
PAT.subject_id,
ADM.hadm_id,
ICU.icustay_id,
ICU_D.first_icu_stay,
ADM.hospital_expire_flag
FROM patients PAT
INNER JOIN admissions ADM
ON PAT.subject_id = ADM.subject_id
LEFT JOIN icustays ICU
ON ICU.subject_id = ADM.subject_id AND ICU.hadm_id = ADM.hadm_id
LEFT JOIN icustay_detail ICU_D
ON ICU_D.icustay_id = ICU.icustay_id
INNER JOIN diagnoses_icd DIA_ICD
ON DIA_ICD.subject_id = ADM.subject_id AND DIA_ICD.hadm_id = ADM.hadm_id AND DIA_ICD.icd9_code in ('99591', '99592', '78552')
WHERE DATE_PART('year', AGE(ADM.admittime, PAT.dob)) > 18
;
"""

sepsis_patients = pd.read_sql_query(sepsis_query, con)
sepsis_patients

,subject_id,hadm_id,icustay_id,first_icu_stay,hospital_expire_flag
0,21,111970,216859.0,True,1
1,38,185910,248910.0,True,0
2,61,189535,217135.0,True,1
3,62,116009,216609.0,True,0
4,64,172056,232593.0,True,0
...,...,...,...,...,...
6125,99865,189346,259297.0,False,1
6126,99865,189346,284772.0,True,1
6127,99912,189380,236183.0,True,0
6128,99985,176670,279638.0,True,0


In [32]:
for first_icu_stay in [True, False]:
    patients_he_1 = [1 if (fis == first_icu_stay) else 0 for fis in sepsis_patients[sepsis_patients.hospital_expire_flag==1].first_icu_stay]
    patients_he_0 = [1 if (fis == first_icu_stay) else 0 for fis in sepsis_patients[sepsis_patients.hospital_expire_flag==0].first_icu_stay]
    t_value, p_value = stats.ttest_ind(
        patients_he_1,
        patients_he_0
    )

    table2_data.append(
        [
            'First Icu Stay = ' + str(first_icu_stay),
            sum(patients_he_1),
            sum(patients_he_0),
            t_value,
            p_value
        ]
    )
table2_data

[['First Hospital Stay = True',
  1329,
  2717,
  0.28581833163806625,
  0.7750269276096289],
 ['First Hospital Stay = False',
  673,
  1396,
  -0.23402245384060752,
  0.8149753645772002],
 ['LOS Hospital',
  18.229534354534355,
  18.24540160601886,
  -0.027154426541734923,
  0.9783374510146332],
 ['First Icu Stay = True',
  1658,
  3640,
  -6.036069295271196,
  1.6715457434754675e-09],
 ['First Icu Stay = False',
  344,
  473,
  6.156334050648287,
  7.916329310853062e-10]]

### LOS ICU

In [33]:
sepsis_query = \
"""
SELECT
DISTINCT
PAT.subject_id,
ADM.hadm_id,
ICU.icustay_id,
ICU_D.los_icu,
ADM.hospital_expire_flag
FROM patients PAT
INNER JOIN admissions ADM
ON PAT.subject_id = ADM.subject_id
LEFT JOIN icustays ICU
ON ICU.subject_id = ADM.subject_id AND ICU.hadm_id = ADM.hadm_id
LEFT JOIN icustay_detail ICU_D
ON ICU_D.icustay_id = ICU.icustay_id
INNER JOIN diagnoses_icd DIA_ICD
ON DIA_ICD.subject_id = ADM.subject_id AND DIA_ICD.hadm_id = ADM.hadm_id AND DIA_ICD.icd9_code in ('99591', '99592', '78552')
WHERE DATE_PART('year', AGE(ADM.admittime, PAT.dob)) > 18
;
"""

sepsis_patients = pd.read_sql_query(sepsis_query, con)
sepsis_patients

,subject_id,hadm_id,icustay_id,los_icu,hospital_expire_flag
0,21,111970,216859.0,8.364722,1
1,38,185910,248910.0,25.548461,0
2,61,189535,217135.0,2.009144,1
3,62,116009,216609.0,2.825718,0
4,64,172056,232593.0,2.325046,0
...,...,...,...,...,...
6125,99865,189346,259297.0,0.821111,1
6126,99865,189346,284772.0,1.991829,1
6127,99912,189380,236183.0,12.086736,0
6128,99985,176670,279638.0,11.299838,0


In [34]:
patients_he_1 = sepsis_patients[(sepsis_patients.hospital_expire_flag==1) & sepsis_patients.los_icu.notna()].los_icu
patients_he_0 = sepsis_patients[(sepsis_patients.hospital_expire_flag==0) & sepsis_patients.los_icu.notna()].los_icu
t_value, p_value = stats.ttest_ind(
        patients_he_1,
        patients_he_0
)

table2_data.append(
    [
        'LOS ICU',
        patients_he_1.mean(),
        patients_he_0.mean(),
        t_value,
        p_value
    ]
)
table2_data

[['First Hospital Stay = True',
  1329,
  2717,
  0.28581833163806625,
  0.7750269276096289],
 ['First Hospital Stay = False',
  673,
  1396,
  -0.23402245384060752,
  0.8149753645772002],
 ['LOS Hospital',
  18.229534354534355,
  18.24540160601886,
  -0.027154426541734923,
  0.9783374510146332],
 ['First Icu Stay = True',
  1658,
  3640,
  -6.036069295271196,
  1.6715457434754675e-09],
 ['First Icu Stay = False',
  344,
  473,
  6.156334050648287,
  7.916329310853062e-10],
 ['LOS ICU',
  7.747044448606949,
  6.939970416182654,
  2.972683559996579,
  0.002963599011663082]]

## Summary - ICU Stays

In [35]:
table2_df = pd.DataFrame(table2_data, columns=['Attribute', 'Hospital Expired == 1', 'Hospital Expired == 0', 't_value', 'p_value'])
table2_df

,Attribute,Hospital Expired == 1,Hospital Expired == 0,t_value,p_value
0,First Hospital Stay = True,1329.000000,2717.000000,0.285818,7.750269e-01
1,First Hospital Stay = False,673.000000,1396.000000,-0.234022,8.149754e-01
2,LOS Hospital,18.229534,18.245402,-0.027154,9.783375e-01
3,First Icu Stay = True,1658.000000,3640.000000,-6.036069,1.671546e-09
4,First Icu Stay = False,344.000000,473.000000,6.156334,7.916329e-10
5,LOS ICU,7.747044,6.939970,2.972684,2.963599e-03


## Univariate Analysis - Interventions

In [36]:
table3_data = []

In [37]:
for table_name in ["crrt_durations", "phenylephrine_durations", "adenosine_durations", "vasopressin_durations", "dobutamine_durations", "vasopressor_durations", "dopamine_durations", "epinephrine_durations", "isuprel_durations", "milrinone_durations", "ventilation_durations", "norepinephrine_durations"]:
    sepsis_query = \
    """
    SELECT
    DISTINCT
    PAT.subject_id,
    ADM.hadm_id,
    ICU.icustay_id,
    ADM.hospital_expire_flag,
    TAB.duration_hours as "duration_hours"
    FROM patients PAT
    INNER JOIN admissions ADM
    ON PAT.subject_id = ADM.subject_id
    INNER JOIN icustays ICU
    ON ICU.subject_id = ADM.subject_id AND ICU.hadm_id = ADM.hadm_id
    INNER JOIN icustay_detail ICU_D
    ON ICU_D.icustay_id = ICU.icustay_id
    INNER JOIN """+table_name+""" TAB
    ON ICU_D.icustay_id = TAB.icustay_id
    INNER JOIN diagnoses_icd DIA_ICD
    ON DIA_ICD.subject_id = ADM.subject_id AND DIA_ICD.hadm_id = ADM.hadm_id AND DIA_ICD.icd9_code in ('99591', '99592', '78552')
    WHERE DATE_PART('year', AGE(ADM.admittime, PAT.dob)) > 18
    ;
    """

    sepsis_patients = pd.read_sql_query(sepsis_query, con)

    patients_he_1 = sepsis_patients[sepsis_patients.hospital_expire_flag==1].duration_hours
    patients_he_0 = sepsis_patients[sepsis_patients.hospital_expire_flag==0].duration_hours
    t_value, p_value = stats.ttest_ind(
        patients_he_1,
        patients_he_0
    )

    table3_data.append(
        [
            table_name,
            patients_he_1.mean(),
            patients_he_0.mean(),
            t_value,
            p_value
        ]
    )
table3_data

[['crrt_durations',
  20.816033508669396,
  22.564838308457713,
  -2.367390479594823,
  0.017976118641913175],
 ['phenylephrine_durations',
  10.380925507900677,
  10.62483816194418,
  -0.6093571283392195,
  0.542307174986574],
 ['adenosine_durations', 0.016666666666666705, 0.016666666666666705, 0.0, 1.0],
 ['vasopressin_durations',
  29.50612269350587,
  26.872538552787663,
  1.5999343473095078,
  0.10975467433675468],
 ['dobutamine_durations',
  12.804551045510456,
  15.637348484848484,
  -1.2247415160043738,
  0.22126227933134518],
 ['vasopressor_durations',
  35.08969817909703,
  23.930028571428572,
  10.914722294292032,
  1.4426512059702128e-27],
 ['dopamine_durations',
  11.514246284501063,
  11.909190031152649,
  -0.39161314123798846,
  0.6953985396609202],
 ['epinephrine_durations',
  8.612709030100332,
  14.917037037037037,
  -3.101117836846942,
  0.002042090936723401],
 ['isuprel_durations', nan, nan, nan, nan],
 ['milrinone_durations',
  26.062477231329687,
  17.280952380952

## Summary - Interventions

In [38]:
table3_df = pd.DataFrame(table3_data, columns=['Attribute', 'Hospital Expired == 1', 'Hospital Expired == 0', 't_value', 'p_value'])
table3_df

,Attribute,Hospital Expired == 1,Hospital Expired == 0,t_value,p_value
0,crrt_durations,20.816034,22.564838,-2.367390,1.797612e-02
1,phenylephrine_durations,10.380926,10.624838,-0.609357,5.423072e-01
2,adenosine_durations,0.016667,0.016667,0.000000,1.000000e+00
3,vasopressin_durations,29.506123,26.872539,1.599934,1.097547e-01
4,dobutamine_durations,12.804551,15.637348,-1.224742,2.212623e-01
5,vasopressor_durations,35.089698,23.930029,10.914722,1.442651e-27
6,dopamine_durations,11.514246,11.909190,-0.391613,6.953985e-01
7,epinephrine_durations,8.612709,14.917037,-3.101118,2.042091e-03
8,isuprel_durations,NaN,NaN,NaN,NaN
9,milrinone_durations,26.062477,17.280952,1.020492,3.082808e-01


## Univariate Analysis - Vitals

In [39]:
table4_data = []

In [40]:
sepsis_query = \
"""
SELECT
DISTINCT
PAT.subject_id,
ADM.hadm_id,
ICU.icustay_id,
VIT.heartrate_mean,
VIT.sysbp_mean,
VIT.diasbp_mean,
VIT.meanbp_mean,
VIT.resprate_mean,
VIT.tempc_mean,
VIT.spo2_mean,
VIT.glucose_mean,
ADM.hospital_expire_flag
FROM patients PAT
INNER JOIN admissions ADM
ON PAT.subject_id = ADM.subject_id
LEFT JOIN icustays ICU
ON ICU.subject_id = ADM.subject_id AND ICU.hadm_id = ADM.hadm_id
LEFT JOIN vitals_first_day VIT
ON VIT.subject_id = ADM.subject_id AND VIT.hadm_id = ADM.hadm_id AND VIT.icustay_id = ICU.icustay_id
INNER JOIN diagnoses_icd DIA_ICD
ON DIA_ICD.subject_id = ADM.subject_id AND DIA_ICD.hadm_id = ADM.hadm_id AND DIA_ICD.icd9_code in ('99591', '99592', '78552')
WHERE DATE_PART('year', AGE(ADM.admittime, PAT.dob)) > 18
;
"""

sepsis_patients = pd.read_sql_query(sepsis_query, con)
# sepsis_patients
for vital in ['heartrate_mean', 'sysbp_mean', 'diasbp_mean', 'meanbp_mean', 'resprate_mean', 'tempc_mean', 'spo2_mean', 'glucose_mean']:
    patients_he_1 = sepsis_patients[(sepsis_patients.hospital_expire_flag==1) & sepsis_patients[vital].notna()][vital]
    patients_he_0 = sepsis_patients[(sepsis_patients.hospital_expire_flag==0) & sepsis_patients[vital].notna()][vital]
    t_value, p_value = stats.ttest_ind(
        patients_he_1,
        patients_he_0
    )

    table4_data.append(
        [
            vital,
            patients_he_1.mean(),
            patients_he_0.mean(),
            t_value,
            p_value
        ]
    )
table4_data

[['heartrate_mean',
  94.47185614009139,
  90.55089575695554,
  8.304312045722385,
  1.2285215885045083e-16],
 ['sysbp_mean',
  106.7248536141622,
  111.56057258796216,
  -11.896612504602855,
  2.832050771605038e-32],
 ['diasbp_mean',
  55.508167072040266,
  58.0802682045279,
  -9.366245259631503,
  1.0395025703215727e-20],
 ['meanbp_mean',
  70.66810877390058,
  73.34352992700421,
  -9.552382705009784,
  1.7987870701314235e-21],
 ['resprate_mean',
  21.849991013206264,
  20.728702910985252,
  8.862700925464447,
  1.0135652770385886e-18],
 ['tempc_mean',
  36.62827305251903,
  36.937632794823465,
  -14.225519208421908,
  3.457485583318761e-45],
 ['spo2_mean',
  95.71118405003476,
  97.07779333870373,
  -14.093240415648902,
  2.122127134737873e-44],
 ['glucose_mean',
  145.51178327215297,
  140.5695064863735,
  3.5471183231519263,
  0.0003924610622258736]]

## Summary - Vitals

In [41]:
table4_df = pd.DataFrame(table4_data, columns=['Attribute', 'Hospital Expired == 1', 'Hospital Expired == 0', 't_value', 'p_value'])
table4_df

,Attribute,Hospital Expired == 1,Hospital Expired == 0,t_value,p_value
0,heartrate_mean,94.471856,90.550896,8.304312,1.228522e-16
1,sysbp_mean,106.724854,111.560573,-11.896613,2.832051e-32
2,diasbp_mean,55.508167,58.080268,-9.366245,1.039503e-20
3,meanbp_mean,70.668109,73.343530,-9.552383,1.798787e-21
4,resprate_mean,21.849991,20.728703,8.862701,1.013565e-18
5,tempc_mean,36.628273,36.937633,-14.225519,3.457486e-45
6,spo2_mean,95.711184,97.077793,-14.093240,2.122127e-44
7,glucose_mean,145.511783,140.569506,3.547118,3.924611e-04
